In [3]:
import cv2
import numpy as np
import os
import json
import ast

In [4]:
pixel_to_um = 0.1155
pixel_to_m = pixel_to_um * 1e-6

In [5]:
# Make a list of all folders in a folder
def get_folder_list(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [16]:
def get_BB(folder, set_list):
    bbox_dict = {}
    for s in set_list[0:3]:
        seq = cv2.VideoCapture(f"{folder}/{s}/Image#%04d.jpg")
        if not seq.isOpened():
            print('error reading image sequence')
            break
        valid, frame = seq.read()
        if not valid:
            print('error reading image')
            break
        bbox = cv2.selectROIs("Select Rois",frame)
        print(bbox)
        bbox = [tuple(x) for x in bbox]
        bbox_dict[s] = bbox

    cv2.destroyAllWindows()
    return bbox_dict

In [7]:
def track(folder, tracker, bbox, output_file):
    '''
    Track an object initially denoted by a bounding box (bbox)
    and write out the location in an output file
    '''
    seq = cv2.VideoCapture(f"{folder}/Image#%04d.jpg")
    if not seq.isOpened():
        print('error reading image sequence')
    valid, frame = seq.read()
    if not valid:
        print('error reading image')

    # Initialize tracker with first frame and bounding box
    valid = tracker.init(frame, bbox)

    data = []
    while True:
        # Read a new frame
        valid, frame = seq.read()
        if not valid:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        valid, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if valid:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        x = [timer]
        x.extend(bbox)
        # data
        data.append(np.array(x))
        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):  # if press SPACE bar
            break
    
    # Save data
    np.savetxt(output_file, data, delimiter=',')
    cv2.destroyAllWindows()

In [8]:
def dict_to_json(dict, filename):
    '''Saves a dictionary to a json file'''
    with open(filename, 'w') as f:
        json.dump(dict, f)

def json_to_dict(filename):
    '''Loads a json file into a dictionary'''
    with open(filename) as f:
        return json.load(f)


In [9]:
def dict_to_txt(dict, filename):
    '''Saves a dictionary to a txt file'''
    with open(filename, 'w') as f:
        for key in dict:
            f.write(f"{key}:{dict[key]}\n")

def txt_to_dict(filename):
    '''Converts a txt file into a dictionary'''
    with open(filename) as f:
        return {line.split(':')[0]: ast.literal_eval(line.split(':')[1][:-1]) for line in f}

# Track the Blobs

In [17]:
# Pre-processing
set_list = get_folder_list('data')
bbox_dict = get_BB('data', set_list)
dict_to_txt(bbox_dict, 'bbox_dict.txt')

[[443 172 237 240]
 [317 445  67  62]
 [818 304  80  89]
 [939  76  92  82]
 [842 606  84  91]]
[[ 240   95  224  154]
 [ 753  207   58   55]
 [ 905  389   70   63]
 [ 925  556   70   68]
 [1070  183   49   56]]
[[ 370  200  142  107]
 [ 447  547  175  157]
 [ 655  489   84   89]
 [ 673  333  110   90]
 [1143  395  118  103]
 [  55  549   75   73]]


In [14]:
# Tracking
bbox_dict = txt_to_dict('bbox_dict.txt')
tracker = cv2.TrackerCSRT_create()

for s in set_list:
    i = 0
    for bbox in bbox_dict[s]:
        print(s, i, bbox)
        tracker = cv2.TrackerCSRT_create()
        output = track(f"data/{s}", tracker, bbox, f"output/{s}-{i}.txt")
        i += 1

set1b 0 (375, 203, 124, 108)
set1b 1 (445, 550, 176, 141)
set1b 2 (653, 504, 73, 74)
set1b 3 (664, 338, 88, 83)
set1b 4 (44, 536, 97, 87)
set1b 5 (1156, 401, 100, 89)
Set2a 0 (619, 178, 86, 79)
Set2a 1 (470, 369, 70, 65)
Set2a 2 (106, 540, 51, 63)
Set2a 3 (877, 631, 127, 123)
set2b 0 (290, 330, 106, 96)
set2b 1 (667, 334, 52, 55)
set2b 2 (1101, 229, 49, 52)
set2b 3 (1022, 505, 47, 63)
Set3a 0 (449, 528, 64, 58)
Set3a 1 (580, 221, 56, 59)
Set3a 2 (879, 639, 138, 126)
set3b 0 (377, 295, 59, 63)
set3b 1 (695, 58, 115, 90)
set3b 2 (902, 420, 57, 62)
Set4a 0 (672, 214, 52, 55)
Set4a 1 (965, 235, 60, 60)
Set4a 2 (1114, 693, 61, 69)
set4b 0 (547, 81, 72, 69)
set4b 1 (1037, 44, 122, 108)
set4b 2 (527, 386, 241, 221)
set4b 3 (350, 671, 59, 48)
Set5a 0 (586, 82, 53, 63)
Set5a 1 (688, 303, 47, 55)
Set5a 2 (666, 630, 57, 62)
set5b 0 (541, 177, 56, 53)
set5b 1 (869, 59, 251, 254)
set5b 2 (954, 529, 92, 74)
set5b 3 (303, 496, 229, 217)
Set6a 0 (43, 294, 59, 71)
Set6a 1 (414, 336, 153, 135)
Set6a 2 (